In [1]:
import pickle

from pyboost import controllers as pb
from pyspark import SparkContext


# sc = SparkContext(master="local[3]")
sc = SparkContext(master=master_url, pyFiles=["dist/pyboost-0.1-py2.7.egg"])
sc.setCheckpointDir('checkpoint/')

In [3]:
import numpy as np


X_train = []
for w in open("../synthetic-data/data/features.txt"):
    X_train.append(np.array(list(map(int, w.split()))))

y_train = []
for w in open("../synthetic-data/data/labels.txt"):
    y_train.append(int(w))

In [4]:
X = sc.parallelize(X_train)
y = sc.parallelize(y_train)

In [5]:
num_round = 5

In [6]:
adaboost = pb.run_adtree_adaboost(sc, y, X, T=num_round, quiet=False)

=== Iteration 1 ===
Min score: 0.905089933524
Purity (farther from 1.0 is better): (2.4999764977275243, 0.44148785068538743)
Predicts (farther from 0.0 is better): (0.45814066546048798, -0.40880238879157499)
Split node: 0 (left)
Split index and value: 2 4.5 

=== Iteration 2 ===
Min score: 0.66351200453
Purity (farther from 1.0 is better): (5.0651465798045221, 0.00024599358974358883)
Predicts (farther from 0.0 is better): (0.81119153851306491, -4.155102540162158)
Split node: 1 (right)
Split index and value: 3 4.5 

=== Iteration 3 ===
Min score: 0.627079161921
Purity (farther from 1.0 is better): (0.31783818255506785, 144.80177700718468)
Predicts (farther from 0.0 is better): (-0.57310644280240775, 2.4876828760126894)
Split node: 0 (left)
Split index and value: 0 4.5 

=== Iteration 4 ===
Min score: 0.124433989993
Purity (farther from 1.0 is better): (2.3294622685617559e-06, 430.28362201694512)
Predicts (farther from 0.0 is better): (-6.4849365515234272, 3.0322222886711145)
Split node:

In [7]:
logitboost = pb.run_adtree_logitboost(sc, y, X, T=num_round, quiet=False)

=== Iteration 1 ===
Min score: 0.865993669366
Purity (farther from 1.0 is better): (2.8971888917078199, 0.51163428855925186)
Predicts (farther from 0.0 is better): (0.53187045959208434, -0.33507259465997591)
Split node: 0 (left)
Split index and value: 2 4.5 

=== Iteration 2 ===
Min score: 0.725009352202
Purity (farther from 1.0 is better): (3.6230292393700063, 0.00024599358974359116)
Predicts (farther from 0.0 is better): (0.64365524118308048, -4.1551025401621535)
Split node: 1 (right)
Split index and value: 3 4.5 

=== Iteration 3 ===
Min score: 0.632127957371
Purity (farther from 1.0 is better): (0.55547517698828774, 80.148331443413312)
Predicts (farther from 0.0 is better): (-0.29396567839528376, 2.1919395304575695)
Split node: 0 (left)
Split index and value: 0 4.5 

=== Iteration 4 ===
Min score: 0.201376951772
Purity (farther from 1.0 is better): (7.3908636442953668e-06, 101.58630074676519)
Predicts (farther from 0.0 is better): (-5.9076329816080557, 2.3104543454418343)
Split nod

# Performance

In [8]:
from pyboost.adtree import run_tree
from pyboost.utils import safe_comp

def test_error(nodes, y, X, max_index=None):    
    error = 0
    margin = 0.0
    for tX, ty in zip(X, y):
        ty = ty + ty - 1
        _m = run_tree(0, nodes, tX, max_index=max_index)
        predict = safe_comp(_m)
        if safe_comp(predict, 0.0) * safe_comp(ty, 0.0) <= 0:
            error += 1
        margin += _m * ty
    return 1.0 - float(error) / len(y), margin / len(y)


def test_all(nodes):
    train_accuracy, train_margin = test_error(nodes, y_train, X_train)
    # test_accuracy, test_margin = test_error(nodes, y_test, X_test)
    print "Train accuracy:", train_accuracy, '\t',
    print "Train margin:", train_margin
    # print "Test accuracy:", test_accuracy, '\t',
    print "Test margin:", test_margin


def test_margin(nodes):
    for max_index in range(num_round):
        accuracy, margin = test_error(nodes, y_train, X_train, max_index=max_index)
        print max_index, accuracy, margin

## Performance of AdaBoost

In [9]:
test_all(adaboost)
test_margin(adaboost)

Train accuracy: 1.0 	Train margin: 13.192174803
Test margin: <function test_margin at 0x7f9b0cf06668>
0 0.5732 -0.10428342082
1 0.6998 0.22071307098
2 0.8228 3.33221609136
3 0.8244 3.79322333331
4 1.0 8.48026542188


# Performance of LogitBoost

In [10]:
test_all(logitboost)
test_margin(logitboost)

Train accuracy: 1.0 	Train margin: 7.89328944568
Test margin: <function test_margin at 0x7f9b0cf06668>
0 0.5732 -0.10428342082
1 0.6998 0.16857136057
2 0.8228 3.28004087369
3 0.8228 3.53317687625
4 1.0 7.80378128392
